# Define Classes and create the SQLite database

> This module defines the classes we use to represent the PKM workflow.

In [ ]:
#| default_exp classdb

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations

In [ ]:
#| export
import json
from enum import Enum
from typing import List, Union, ClassVar
from dataclasses import dataclass
from pydantic import BaseModel, field_serializer, field_validator, Field
from fastlite import *
from fastcore.test import *

We use `from __future__ import annotations` to support forward references in type hints. To be precise in the `@classmethod` we create to keep track of all instances of the class.

## Enum Classes

First we define the possible values of the different variables that are available in the classes. We use the module `enum` to define **Enumerations**. We use this to bind the possible values to a variable name, making the code more readable and maintainable.

In [ ]:
#| export
class InformationType(Enum):
    """Information content types that flow through the PKM workflow."""
    BOOK = "book"
    RESEARCH_PAPER = "research_paper"
    DOCUMENT = "document"
    ANNOTATION = "annotations&highlights"
    NOTE = "note"
    EMAIL = "email"
    DISCORD_MESSAGE = "discord_message"
    WEB_ARTICLE = "web_article"
    YOUTUBE_VIDEO = "youtube_video"
    PODCAST = "podcast"
    PRODUCT_IDEA = "product_idea"
    PROJECT_IDEA = "project_idea"

class Method(Enum):
    """How actions are performed - manually or automatically."""
    MANUAL = "manual"
    AUTOMATIC = "automatic"

class Phase(Enum):
    """The five phases of the PKM workflow."""
    COLLECT = "collect"
    RETRIEVE = "retrieve"
    CONSUME = "consume"
    EXTRACT = "extract"
    REFINE = "refine"

class PhaseQuality(Enum):
    """Quality rating for how well a tool performs in each phase."""
    NA = "na"
    BAD = "bad"
    OK = "ok"
    GREAT = "great"

class OrganizationSystem(Enum):
    """How tools organize and structure information."""
    TAGS = "tags"
    FOLDERS = "folders"
    LINKS = "links"
    JOHNNY_DECIMAL = "johnny_decimal"

In [ ]:
Phase("refine")

<Phase.REFINE: 'refine'>

## PKM Workflow Classes

Next we create a dataclass for each item we need to be present in the PKM workflow.

#### Track instances of classes

We also want to keep track of the instances available for each class. Therefore we need some higher order magic.

- a list in the class to store the instances
- a __init__ method to add the instance to the list
- a classmethod to get the list of instances

We can't just add a `_instances = []` statement to the Class, because Pydantic will then assume it is a model field (private attribute). We need to tell Pydantic to ignore the _instances class variable as a model field and treat is as a class variable. Therefore we need to import `ClassVar` from `typing` and use it to type the _instances variable.

#### Using Pydantic with MiniDataAPI and SQLite

We want to use `Pydantic` Dataclasses to enable typechecking and validation. We also want to use the Dataclasses with the `MiniDataAPI` to create the tables in the `SQLite` database. But `SQLite` only has datatypes: `NULL`, `INTEGER`, `REAL`, `TEXT`, and `BLOB`. So no `list` or any of the Dataclass(Enum) types we use.

To be able to use both `Pydanctic` and the `MiniDataAPI` we will do two things:

1. Define a Pydantic Dataclass with the correct datatypes and a Dataclass that has the same fields as the Pydantic Dataclass, but with datatypes that can be used with SQLite.
2. We add `@field_serializer` and `@field_validator` methods to the Pydantic Dataclass that convert the fields to JSON strings when we use the method `.model_dump()` on the instance of the Pydantic Dataclass.

This way we can:

- create the SQLite database tables using the regular Dataclasses.
- create instances with the Pydantic Dataclass to have easy typechecking and validation.
- convert this instances to `MiniDataAPI` and `SQLite` friendly datatypes using `.model_dump()` on the instance.

```python
class InformationItem(BaseModel):
    info_type: InformationType
    method: list[Union[Method, None]]
    toolflow: list[str]
    
    # Convert Enum to string and list to JSON string so we can add to SQLite
    @field_serializer('info_type', 'method', 'toolflow')
    def serialize_lists(self, v):
        if isintance(v, list):
            return json.dumps([i.value if hasattr(i, 'value') else i for i in v])
        return str(v.value) if hasattr(v, 'value') else str(v)
    
    # Convert JSON string from SQLite to list of Enum and strings
    @field_validator('method', 'toolflow', mode='before')
    def parse_json_lists(cls, v):
        if isinstance(v, str):
            return json.loads(v)
        return v
```

**Pydantic Dataclasses**

Used for typechecking.

When creating a new instance for an `InformationItem` the `toolflow` must be given as a list of `Tool` objects. The typechecking makes sure that any `Tool` object mentioned in the `toolfow` list, does exist as an actual `Tool` instance. So make sure to first create  all the `Tool` instances that are needed for an `InformationItem`, before creating the `InformationItem` instance.

:::{.callout-tip}
I had some serious trouble getting the Pydantic dataclass validations to work. One of the issues is described above and is about SQLite not supporting all datatypes. A second major issue is that the Pydantic Dataclasses reference each other. The `InformationItem` references the `Tool` in the `toolflow` field. I would also be convenient to store all the `InformationItem`s that can be used with a certain `Tool`, but in that case we would create a circular reference between `InformationItem` and `Tool`.

We decided to remove the `information_items` list from `Tool`. When we need to get all the `InformationItem`s that are supported by a `Tool` we can write a Python function or do a SQL-query on the SQLite database.

But then we are left with the fact that we want a list of `Tool`s that exist. These are the options considered:

- `toolflow: list[Tool]`
- `toolflow: list[Tool.name]`
- `toolflow: list[str]`

The last option is used in combination with validation to ensure each string is a valid Tool.name.

Here's why this is the best approach:

- Clean serialization (no complex object embedding)
- Human-readable in the database
- Type safety through validation
- Easy to query

The same goes for the `Improvement` class and the field `tool`.
:::

In [ ]:
#| export
class InformationItem(BaseModel):
    """Represents an information item flowing through the PKM workflow."""
    name: str = Field(..., description="Name of the information item")
    info_type: InformationType = Field(..., description="Type of information item, e.g. book, article, video, etc.")
    method: list[Union[Method, None]] = Field(..., description="Methods used at each phase in order: collect, retrieve, consume, extract, refine")
    toolflow: list[Union[str, list[str], tuple[str], None]] = Field(..., description="Tools used for this item at each phase in order: collect, retrieve, consume, extract, refine")

    _instances: ClassVar[List[InformationItem]] = []

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances.append(self)
    
    @classmethod
    def get_instances(cls) -> list[InformationItem]:
        return cls._instances.copy()
    
    @field_serializer('info_type','method', 'toolflow')
    def db_serialize(self, v):
        if isinstance(v, list):
            return json.dumps([i.value if hasattr(i, 'value') else i for i in v])
        return str(v.value) if hasattr(v, 'value') else v
    
    @field_validator('info_type','method', 'toolflow', mode='before')
    def parse_json_lists(cls, value):
        if isinstance(value, str):
            return json.loads(value)
        return value
    
    @field_validator('toolflow')
    def validate_tool_names(cls, v):
        if len(v) != 5:
            raise ValueError(f"Toolflow must have 5 tools, got {len(v)}")
        valid_tools = {tool.name for tool in Tool.get_instances()}
        for p in v: # Phase-tools
            if p is None:
                continue
            elif isinstance(p, str): # Case of single tool in phase
                if p not in valid_tools: raise ValueError(f"Tool '{p}' does not exist")
            elif isinstance(p, (list, tuple)): # Case of multiple tools in phase
                for t in p:
                    if t not in valid_tools: raise ValueError(f"Tool '{t}' does not exist")
            else:
                raise ValueError(f"Tool '{p}' is not a string or list of strings or tuple of strings")

        return v


class Tool(BaseModel):
    """Represents a PKM tool with information on the supported OrganizationSystems and for each Phase the perceived quality."""
    name: str = Field(..., description="Name of the tool")
    organization_system: list[OrganizationSystem] = Field(..., description="Organization systems supported by the tool")
    phase_quality: list[PhaseQuality] = Field(..., description="Quality of the tool for each phase in order: collect, retrieve, consume, extract, refine")

    _instances: ClassVar[List[Tool]] = []

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances.append(self)
    
    @classmethod
    def get_instances(cls) -> list[Tool]:
        return cls._instances.copy()
    
    @field_serializer('organization_system', 'phase_quality')
    def db_serialize(self, v):
        if isinstance(v, list):
            return json.dumps([i.value if hasattr(i, 'value') else i for i in v])
        return str(v.value) if hasattr(v, 'value') else v
    
    @field_validator('organization_system', 'phase_quality', mode='before')
    def parse_json_lists(cls, value):
        if isinstance(value, str):
            return json.loads(value)
        return value
    
    @field_validator('phase_quality')
    def validate_phase_quality(cls, v):
        if len(v) != 5:
            raise ValueError(f"Phase quality must have 5 phases, got {len(v)}")
        return v
    

class Improvement(BaseModel):
    """Tracks workflow improvements needed for better PKM effectiveness."""
    title: str = Field(..., description="Title of the improvement")
    what: str = Field(..., description="What needs to be improved")
    why: str = Field(..., description="Why is this improvement needed")
    prio: int = Field(..., description="Priority of the improvement")
    tool: str = Field(..., description="Tool that needs improvement")
    phase: Phase = Field(..., description="Phase that needs improvement")

    _instances: ClassVar[List[Improvement]] = []

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances.append(self)
    
    @classmethod
    def get_instances(cls) -> list[Improvement]:
        return cls._instances.copy()
    
    @field_serializer('tool', 'phase')
    def db_serialize(self, v):
        if isinstance(v, list):
            return json.dumps([i.value if hasattr(i, 'value') else i for i in v])
        return str(v.value) if hasattr(v, 'value') else v
    
    @field_validator('phase', mode='before')
    def parse_json_lists(cls, value):
        if isinstance(value, str):
            return json.loads(value)
        return value
    
    @field_validator('tool')
    def validate_tool_names(cls, v):
        valid_tools = {tool.name for tool in Tool.get_instances()}
        if v not in valid_tools:
            raise ValueError(f"Tool '{tool_name}' does not exist")
        return v

Test creating instances

In [ ]:
reader = Tool(name="reader", organization_system=[OrganizationSystem.TAGS], phase_quality=[PhaseQuality.GREAT, PhaseQuality.OK, PhaseQuality.OK, PhaseQuality.OK, PhaseQuality.OK])
obsidian = Tool(name="obsidian", organization_system=[OrganizationSystem.TAGS], phase_quality=[PhaseQuality.GREAT, PhaseQuality.BAD, PhaseQuality.BAD, PhaseQuality.BAD, PhaseQuality.BAD])
inf_a = InformationItem(name="infoitem_a", info_type=InformationType.BOOK, method=[Method.MANUAL], toolflow=[("reader", "obsidian"), "obsidian", "reader", "obsidian", "reader"])
imp_a = Improvement(title="improvement_a", what="gras", why="dus", prio=0, tool="reader", phase=Phase.COLLECT)

Test creating list of instances

In [ ]:
test_eq(len(Improvement.get_instances()), 1)
test_eq(len(Tool.get_instances()), 2)

**Regular Dataclasses with SQLite datatypes**

Used for creating the tables in the SQLite database.
These contain the same fields as the Pydantic Dataclasses we defined above. But these Dataclasses only contain datatypes that are supported by SQLite and have an `id: int` field added as a primary key and use the build-in `@dataclass` decorator, because `FastLite` doesn't support Pydantic Dataclasses.

In [ ]:
#| export
@dataclass
class ImprovementDB:
    id: int
    title: str
    what: str
    why: str
    prio: int
    tool: str
    phase: str

@dataclass
class InformationItemDB:
    id: int
    name: str
    info_type: str
    method: str
    toolflow: str

@dataclass
class ToolDB:
    id: int
    name: str
    organization_system: str
    phase_quality: str

In [ ]:
tre = [("koe", "honderd"), None, "hond"]


In [ ]:
"hond" in tre

True

In [ ]:
"koe" in tre

False

## SQLite database

Connect to the database in the `main.py`. We should also enable foreign key constraints. These are disabled by default in Sqlite.

For testing purposes in this module we will use `db = database(":memory:")` to create an in-memory database.

In [ ]:
#| export
def create_db(loc="static/infoflow.db"):
    db = database(loc)
    db.execute("PRAGMA foreign_keys = ON;")
    inf_tbl = db.create(InformationItemDB)
    tool_tbl = db.create(ToolDB)
    impr_tbl = db.create(ImprovementDB)
    return db, inf_tbl, tool_tbl, impr_tbl


::: {.callout-tip}
We can add foreign key constraints to the tables using the `transform` method from `sqlite_utils`.

```python
inf_tbl.transform(add_foreign_keys=[("<field_name>", "<table_name_to_connect>", "<field_name_in_table_to_connect>")])
```
:::

But for now we won't use foreign key constraints.

Tests and usage examples

In [ ]:
db, inf_tbl, tool_tbl, imp_tbl = create_db(":memory:")

In [ ]:
inf_tbl.columns

[Column(cid=0, name='id', type='INTEGER', notnull=0, default_value=None, is_pk=1),
 Column(cid=1, name='name', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=2, name='info_type', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=3, name='method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=4, name='toolflow', type='TEXT', notnull=0, default_value=None, is_pk=0)]

Add the previously created instances to the tables

In [ ]:
reader.model_dump()

{'name': 'reader',
 'organization_system': '["tags"]',
 'phase_quality': '["great", "ok"]'}

In [ ]:
inf_a.model_dump()

{'name': 'infoitem_a',
 'info_type': 'book',
 'method': '["manual"]',
 'toolflow': '["reader", "obsidian"]'}

In [ ]:
imp_a.model_dump()

{'title': 'improvement_a',
 'what': 'gras',
 'why': 'dus',
 'prio': 0,
 'tool': 'reader',
 'phase': 'collect'}

In [ ]:
tool_tbl.insert(reader.model_dump())
tool_tbl.insert(obsidian.model_dump())
inf_tbl.insert(inf_a.model_dump())
imp_tbl.insert(imp_a.model_dump())

ImprovementDB(id=1, title='improvement_a', what='gras', why='dus', prio=0, tool='reader', phase='collect')

Now retrieve the info from the database as intances from the Pydantic Dataclass

**Method 1:**

In [ ]:
db.t

improvement_db, information_item_db, tool_db

In [ ]:
db.t.tool_db()

[{'id': 1,
  'name': 'reader',
  'organization_system': '["tags"]',
  'phase_quality': '["great", "ok"]'},
 {'id': 2,
  'name': 'obsidian',
  'organization_system': '["tags"]',
  'phase_quality': '["great", "bad"]'}]

In [ ]:
reader_back_from_sqlite_1 = Tool(**db.t.tool_db()[0])
reader_back_from_sqlite_1

Tool(name='reader', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=[<PhaseQuality.GREAT: 'great'>, <PhaseQuality.OK: 'ok'>])

**Method 2:**

In [ ]:
tool_tbl()

[ToolDB(id=1, name='reader', organization_system='["tags"]', phase_quality='["great", "ok"]'),
 ToolDB(id=2, name='obsidian', organization_system='["tags"]', phase_quality='["great", "bad"]')]

In [ ]:
tool_tbl()[0]

ToolDB(id=1, name='reader', organization_system='["tags"]', phase_quality='["great", "ok"]')

In [ ]:
reader_back_from_sqlite_2 = Tool(**tool_tbl()[0].__dict__)
reader_back_from_sqlite_2

Tool(name='reader', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=[<PhaseQuality.GREAT: 'great'>, <PhaseQuality.OK: 'ok'>])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()